### Finding the vol fitting outliers
   * produce the ATM option spread in 2021
   * find <20 periods of spikes
   * go to CT to check the vol fitting curves
   * write down the summaries into confluence

In [ ]:
from cdcqr.data.deribit.data_utils import quoto_and_trade_analysis
from datetime import datetime
import sys
import pandas as pd
from cdcqr.data.deribit.data_utils import get_spread_from_ATM_option_quote_data
from cdcqr.common.utils import parallel_jobs
from cdcqr.common.config import LOCAL_DATA_DIR
sys.path.append('C://github/cryptoderiv-quant/')
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
#from ct.utils import plot2

quoto_and_trade_analysis?

In [ ]:
ret_dict, ret_df = quoto_and_trade_analysis(contract='BTC-25JUN21-60000-C', date='2021-04-18')

In [ ]:
ret_df['strike'].plot()

### 1) Load quote data, finding the ATM option, produce the spread at minutes level

In [ ]:
def get_spread_from_ATM_option_quote_data(date):
    """
    load quote data -> find the ATM call -> calcuate spread, resample at 1 min level -> return the max spread time series
    """
    df = data_loader(exchange='deribit', date = date.strftime('%Y-%m-%d'), data_type='quotes', symbol='OPTIONS', )
    df1 = df[['symbol','timestamp','ask_price','bid_price']]
    df2 = df1[df1['symbol'].str.contains('BTC') & df1['symbol'].str.contains('-C')]
    df2['spread'] = df2['ask_price'] - df2['bid_price']
    df3 = df2[['symbol','timestamp','spread']]
    ATM_call = df3.groupby('symbol')['spread'].count().sort_values(ascending=False).index[0]
    df4 = df3.query('symbol==@ATM_call')
    df4['dt'] = pd.to_datetime(df4['timestamp'], unit='us')
    df5 = df4.set_index('dt').resample('1T')['spread'].max()
    df5 = df5[df5.index>=date]
    return df5

In [ ]:
from cdcqr.data.dataloader import data_loader
dates = pd.date_range(datetime(2021,1,1), datetime(2021,10,28))
date = dates[0]

In [ ]:
get_spread_from_ATM_option_quote_data(date)

### 2) run the above function in parallel

In [ ]:
dates = pd.date_range(datetime(2021,1,1), datetime(2021,4,1))
dates2 = pd.date_range(datetime(2021,4,2), datetime(2021,7,1))
dates3 = pd.date_range(datetime(2021,7,2), datetime(2021,10,27))


In [ ]:
ret_dict = parallel_jobs(get_spread_from_ATM_option_quote_data, dates3)

In [ ]:
ret_df = pd.concat(ret_dict.values()).sort_index()

In [ ]:
ret_df_08_10= ret_df.copy()

In [ ]:
ret_df_04_07 = ret_df.copy()

In [ ]:
ret_df.plot()

In [ ]:
ret_df_01_03 = ret_df.copy()

In [ ]:
ret_df_04_07.plot()

In [ ]:
df_2021[(df_2021['dt']>=datetime(2021,7,8,0,0)) & (df_2021['dt']<datetime(2021,7,9,0,0))].set_index('dt')['spread'].plot()

In [ ]:
df_2021[(df_2021['dt']>=datetime(2021,5,23,0,0)) & (df_2021['dt']<datetime(2021,5,24,0,0))].set_index('dt')['spread'].plot()

In [ ]:
ret_df_04_07.shape

In [ ]:
ret_df_08_10.shape

In [ ]:
df_2021 = ret_df_01_03.append(ret_df_04_07).append(ret_df_08_10)

In [ ]:
df_2021['date'] = df_2021['dt'].dt.date

In [ ]:
df_2021.to_pickle()

### 3) find the spike periods

In [ ]:
df_2021.plot()

In [ ]:
df_2021 = df_2021.reset_index()

#### spikes >=0.5

In [ ]:
spikes_lv1 = df_2021[df_2021>=0.5].reset_index()

In [ ]:
spikes_lv1['date'] = spikes_lv1['dt'].dt.date

In [ ]:
spikes_lv1.groupby('date').count()

In [ ]:
df_2021[(df_2021['dt']>=datetime(2021,9,27,0,0)) & (df_2021['dt']<datetime(2021,9,28,0,0))].set_index('dt')['spread'].plot()

### on 20210519, check ATM bid/ask price, quantity and flow for 40k, 32k, 48k put/call with expire on 20211231 

In [ ]:
ret_dict, ret_df_40C = quoto_and_trade_analysis(contract='BTC-31DEC21-40000-C', date='2021-05-19')

In [ ]:
ret_dict, ret_df_40P = quoto_and_trade_analysis(contract='BTC-31DEC21-40000-P', date='2021-05-19')

In [ ]:
ret_dict, ret_df_32P = quoto_and_trade_analysis(contract='BTC-31DEC21-32000-P', date='2021-05-19')

In [ ]:
ret_dict, ret_df_32C = quoto_and_trade_analysis(contract='BTC-31DEC21-32000-C', date='2021-05-19')

In [ ]:
ret_dict, ret_df_48C = quoto_and_trade_analysis(contract='BTC-31DEC21-48000-C', date='2021-05-19')

In [ ]:
ret_dict, ret_df_48P = quoto_and_trade_analysis(contract='BTC-31DEC21-48000-P', date='2021-05-19')

### 40K CALL

In [ ]:
df2plot = ret_df_40C.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'] = df2plot['flow'].replace(0, np.nan)
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'], linestyle='None', marker = 'x')
plt.title('BTC-31DEC21-40000-C')
plt.show()

In [ ]:
ret_df_40C.head(2)

In [ ]:
df2plot = ret_df_40C.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_amount','bid_amount','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_amount','bid_amount',]].plot(ax=ax1, linestyle='None', marker = '.')
plt.show()

### 40K PUT

In [ ]:
df2plot = ret_df_40P.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'] = df2plot['flow'].replace(0, np.nan)
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'], linestyle='None', marker = 'x')
plt.title('BTC-31DEC21-40000-P')
plt.show()

In [ ]:
df2plot = ret_df_40P.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_amount','bid_amount','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_amount','bid_amount',]].plot(ax=ax1, linestyle='None', marker = '.')
plt.title('BTC-31DEC21-40000-P')
plt.show()

#### 32K CALL

In [ ]:
df2plot = ret_df_32C.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'] = df2plot['flow'].replace(0, np.nan)
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'], linestyle='None', marker = 'x')
plt.title('BTC-31DEC21-32000-C')
plt.show()

In [ ]:
df2plot = ret_df_32C.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_amount','bid_amount','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_amount','bid_amount',]].plot(ax=ax1, linestyle='None', marker = '.')
plt.title('BTC-31DEC21-32000-C')
plt.show()

#### 32K PUT

In [ ]:
df2plot = ret_df_32P.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'] = df2plot['flow'].replace(0, np.nan)
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'], linestyle='None', marker = 'x')
plt.title('BTC-31DEC21-32000-P')
plt.show()

In [ ]:
df2plot = ret_df_32P.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_amount','bid_amount','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_amount','bid_amount',]].plot(ax=ax1, linestyle='None', marker = '.')
plt.title('BTC-31DEC21-32000-P')
plt.show()

#### 48K CALL

In [ ]:
df2plot = ret_df_48C.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
#df2plot = ret_df_48P.set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'], linestyle='None', marker = 'x')
plt.title('BTC-31DEC21-48000-P')
plt.show()

In [ ]:
df2plot = ret_df_48C.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_amount','bid_amount','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_amount','bid_amount',]].plot(ax=ax1, linestyle='None', marker = '.')
plt.title('BTC-31DEC21-48000-C')
plt.show()

#### 48K PUT

In [ ]:
df2plot = ret_df_48P.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_price','bid_price','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_price','bid_price',]].plot(ax=ax1, linestyle='None', marker = '.')
ax2 = ax1.twinx()
df2plot['flow'].plot(ax=ax2, legend=['trade'], color = ['b'], linestyle='None', marker = 'x')
plt.title('BTC-31DEC21-48000-P')
plt.show()

In [ ]:
df2plot = ret_df_48P.query('timestamp_dt>=datetime(2021,5,19,16,25)').query('timestamp_dt<=datetime(2021,5,19,16,40)').set_index('timestamp_dt')[['ask_amount','bid_amount','flow']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))
df2plot[['ask_amount','bid_amount',]].plot(ax=ax1, linestyle='None', marker = '.')
plt.title('BTC-31DEC21-48000-P')
plt.show()

#### PERP

#### OERDERBOOK

In [ ]:
[x for x in list(df['symbol'].unique()) if 'BTC-31DEC21' in x]

In [ ]:
quoto_and_trade_analysis??

In [ ]:
exchange = 'deribit'
date = '2021-05-19'
symbol = 'OPTIONS'
df_opt_trade = data_loader(exchange, date,"trades",symbol)

In [ ]:
df_opt_trade = df_opt_trade.query('symbol=="BTC-31DEC21-40000-P"')

In [ ]:
df_opt_trade['dt'] = pd.to_datetime(df_opt_trade['timestamp'], unit='us') # check convert ts

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
df_opt_trade.set_index('dt')

In [ ]:
ret_df_48P.query('timestamp_dt>=datetime(2021,5,19,15,00)').query('timestamp_dt<=datetime(2021,5,19,16,00)')

In [ ]:
ret_df_40P.query('timestamp_dt>=datetime(2021,5,19,15,)').query('timestamp_dt<=datetime(2021,5,19,16,)')

### check order book snapshot

In [ ]:
exchange = 'deribit'
date = '2021-05-19'
symbol = 'OPTIONS'
data_type = 'book_snapshot_25'
df_snapshot = data_loader(exchange, date,data_type,symbol)

In [ ]:
df_snapshot.head()

In [ ]:
optchain1Min_20210519_20211231 = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain1Min_20210519_20211231.pkl'))

In [ ]:
optchain1Min_20210519_20211231['underly'].unique()

In [ ]:
optchain1Min_20210519_20211231.query('underly=="BTC-31DEC21"')['s'].plot()

In [ ]:
optchain1Min_20210519_20211231.query('underly=="BTC-31DEC21"').query('symbol=="BTC-31DEC21-32000-C"').query('tm>=datetime(2021,5,19,16,25)').query('tm<=datetime(2021,5,19,16,40)')['s'].plot(title='32k')

In [ ]:
optchain1Min_20210519_20211231.query('underly=="BTC-31DEC21"').query('symbol=="BTC-31DEC21-40000-C"').query('tm>=datetime(2021,5,19,16,25)').query('tm<=datetime(2021,5,19,16,40)')['s'].plot(title='40k')

In [ ]:
optchain1Min_20210519_20211231.query('underly=="BTC-31DEC21"').query('symbol=="BTC-31DEC21-48000-C"').query('tm>=datetime(2021,5,19,16,25)').query('tm<=datetime(2021,5,19,16,40)')['s'].plot(title='48k')

### time graph for all strikes near ATM of (bid+ask) amount , and on axis2  spot price and underlying futures price

In [ ]:
## data 1: optchain
optchain1Min_20210519_20211231 = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain1Min_20210519_20211231.pkl'))

In [ ]:
# data 2: quote and trade
ret_df_40C.head()

In [ ]:
# data 3: PERP
exchange = 'deribit'
date = '2021-05-19'
symbol = 'BTC-PERPETUAL'
data_type = 'quotes'
df_perp = data_loader(exchange, date,data_type,symbol)

In [ ]:
df_perp['dt'] = pd.to_datetime(df_perp['timestamp'], unit='us')
df_perp['mid'] = (df_perp['ask_price']+df_perp['bid_price'])/2
df_perp = df_perp.set_index('dt')
df_perp_1min = df_perp.resample('1min').mean()['mid']

In [ ]:
ret_df_40C_ext = ret_df_40C.set_index('timestamp_dt').reindex(optchina_btc_c.index, method='ffill').join(optchina_btc_c, lsuffix='_left', rsuffix='_right')

ret_df_40C_ext2 = ret_df_40C_ext.join(df_perp_1min)

df2plot = ret_df_40C_ext2[['total_amount','s','mid']]
fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))

df2plot[['total_amount']].plot(ax=ax1,  linestyle='None', marker = '.', color=['r'])
ax1.legend(loc="upper left")
ax2 = ax1.twinx()
df2plot[['s','mid']].plot(ax=ax2,)
ax2.legend(['underlying','PERP'], loc="upper right")
plt.title('20210519 BTC 40K call expire=20211231 v.s. PERP and underlying')
plt.show()

In [ ]:
ret_df_40C['total_amount'] = ret_df_40C['ask_amount'] + ret_df_40C['bid_amount']

In [ ]:
ret_df_40C.set_index('timestamp_dt')['total_amount'].plot(linestyle='None', marker = '.')

In [ ]:
for df_quote_str, symbol in zip(['32C','32P','40C','40P','48C','48P'],['BTC-31DEC21-32000-C', 'BTC-31DEC21-32000-P', 'BTC-31DEC21-40000-C', 'BTC-31DEC21-40000-P', 'BTC-31DEC21-48000-C', 'BTC-31DEC21-48000-P']):
    print(df_quote_str, symbol)
    df_quote = eval('ret_df_{}'.format(df_quote_str))
    df_quote['total_amount'] = df_quote['ask_amount']+df_quote['bid_amount']
    optchain = optchain1Min_20210519_20211231.query('underly=="BTC-31DEC21"').query('symbol==@symbol')
    
    df_quote_ext = df_quote.set_index('timestamp_dt').reindex(optchain.index, method='ffill').join(optchain, lsuffix='_left', rsuffix='_right')

    df_quote_ext2 = df_quote_ext.join(df_perp_1min)

    df2plot = df_quote_ext2[['total_amount','s','mid']]
    fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))

    df2plot[['total_amount']].plot(ax=ax1,  linestyle='None', marker = '.', color=['r'])
    ax1.legend(loc="upper left")
    ax2 = ax1.twinx()
    df2plot[['s','mid']].plot(ax=ax2,)
    ax2.legend(['underlying','PERP'], loc="upper right")
    plt.title('20210519 BTC {} expire=20211231 v.s. PERP and underlying'.format(df_quote_str))
    plt.show()
    

### put above plots into a single plot
   * using different marker size to represent amount of bid/ask
   * using diffrernt color to mark bid and ask 
   * left y-axis for strike
   * right y-axis for underlying price
   * hover mouse to show option price

In [ ]:
df_quote = ret_df_40C
df_quote['total_amount'] = df_quote['ask_amount']+df_quote['bid_amount']
optchain = optchain1Min_20210519_20211231.query('underly=="BTC-31DEC21"').query('symbol==@symbol')

df_quote_ext = df_quote.set_index('timestamp_dt').reindex(optchain.index, method='ffill').join(optchain, lsuffix='_left', rsuffix='_right')
df_quote_ext2 = df_quote_ext.join(df_perp_1min)
df2plot = df_quote_ext2[['total_amount','s','mid']]


fig, ax1 = plt.subplots(1,1, figsize=(8*4,6))

df2plot[['total_amount']].plot(ax=ax1,  linestyle='None', marker = '.', color=['r'])
ax1.legend(loc="upper left")

ax2 = ax1.twinx()
df2plot[['s','mid']].plot(ax=ax2,)
ax2.legend(['underlying','PERP'], loc="upper right")


plt.title('20210519 BTC {} expire=20211231 v.s. PERP and underlying'.format(df_quote_str))
plt.show()

In [ ]:
import plotly.express as px
df = px.data.gapminder()
fig = px.scatter(df.query("year==2007"), x="gdpPercap", y="lifeExp", size="pop", color="continent",
           hover_name="country", log_x=True, size_max=60)
fig.show()

In [ ]:
df2plot = df2plot.reset_index()

In [ ]:
df2plot['strike'] = 40000
df2plot['strike2'] = 40000

In [ ]:
df2plot

In [ ]:
fig = px.scatter(df2plot, x="index", y="strike", size="total_amount",  hover_name="mid")
#fig = px.scatter(df2plot, x="index", y="strike", size="total_amount",  hover_name="mid")
fig.show()

In [ ]:
ret_df_40C_reduced = ret_df_40C[['ask_amount','ask_price','bid_price','bid_amount','timestamp_dt']]

In [ ]:
ret_df_40C_reduced['dt'] =  pd.to_datetime(ret_df_40C_reduced['timestamp_dt'], format='%Y')

In [ ]:
## resample time
## catagoricaize columns 
ret_df_40C_reduced['dt'].resample('3T')

In [ ]:
df_quote = ret_df_40C_reduced.copy()
optchain = optchain1Min_20210519_20211231.query('underly=="BTC-31DEC21"').query('symbol==@symbol')

df_quote_ext = df_quote.set_index('timestamp_dt').reindex(optchain.index, method='ffill').join(optchain, lsuffix='_left', rsuffix='_right')

df_quote_ext2 = df_quote_ext.join(df_perp_1min)

In [ ]:
df_quote_ext2.head(2)

In [ ]:
df2plot = df_quote_ext2[['ask_amount','ask_price','bid_price','bid_amount','tm','strike','s','mid']]

In [ ]:
df2plot = df2plot[~df2plot.duplicated()]

In [ ]:
df2plot.head()

In [ ]:
df2plot_ask = df2plot[['ask_amount','ask_price','tm','strike','s','mid']]
df2plot_ask['type'] = 'ask'
df2plot_ask.columns = ['amount','price','tm','strike','s','mid','type']

df2plot_bid = df2plot[['bid_amount','bid_price','tm','strike','s','mid']]
df2plot_bid['type'] = 'bid'
df2plot_bid.columns = ['amount','price','tm','strike','s','mid','type']


In [ ]:
df2plot_new = df2plot_ask.append(df2plot_bid).reset_index()

In [ ]:
df2plot_new[~df2plot_new.duplicated()]

In [ ]:
from plotly.subplots import make_subplots
subfig = make_subplots(specs=[[{"secondary_y": True}]])
fig = px.scatter(df2plot_new, x="index", y="strike", size="amount",  hover_name="price", color="type")

fig.show()

In [ ]:
fig2 = px.line(df2plot_new.query('type=="ask"'), x="index", y=['s','mid'])
fig2.show()

In [ ]:
import plotly.graph_objects as go

fig3 = go.Figure(data=fig.data + fig2.data)

In [ ]:
fig3.show()

In [ ]:
fig3.write_html("file.html")